In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import random
import matplotlib.pyplot as plt
import numpy as np
# Import requests library.
import requests
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the API key.
from config import g_key
from config import weather_api_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description,Rain (inches),Snow (inches)
0,0,Turinsk,58.05,63.70,48.24,98,100,9.51,RU,2020-06-28 16:17:28,moderate rain,2.14,0.0
1,1,Berbera,10.44,45.01,98.04,33,24,11.45,SO,2020-06-28 16:17:28,few clouds,0.00,0.0
2,2,Kapaa,22.08,-159.32,73.40,94,75,4.70,US,2020-06-28 16:17:28,moderate rain,1.02,0.0
3,3,Adre,13.47,22.20,93.85,25,90,1.07,TD,2020-06-28 16:17:07,overcast clouds,0.00,0.0
4,4,Omboue,-1.57,9.26,76.17,74,20,10.76,GA,2020-06-28 16:17:28,few clouds,0.00,0.0


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Ask the customer to if they want it to be raining or snowing?.
rain = (input("Do you want it to be raining? (yes/no) "))
snow = (input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.

if rain == "no" and snow == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                       (city_data_df["Snow (inches)"] == 0)]
    
elif rain == "no" and snow == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                       (city_data_df["Snow (inches)"] > 0)]
elif rain == "yes" and snow == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& \
                                       (city_data_df["Rain (inches)"] > 0) & \
                                       (city_data_df["Snow (inches)"] == 0)]
    
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& \
                                       (city_data_df["Rain (inches)"] > 0) & \
                                       (city_data_df["Snow (inches)"] > 0)]

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Omboue,GA,76.17,-1.57,9.26,
6,Butaritari,KI,82.67,3.07,172.79,
9,Mogadishu,SO,78.84,2.04,45.34,
10,Puerto Del Rosario,ES,78.80,28.50,-13.86,
15,Cabo San Lucas,MX,84.20,22.89,-109.91,
27,Veyrier,CH,80.60,46.17,6.18,
35,Arraial Do Cabo,BR,75.20,-22.97,-42.03,
41,Puerto Escondido,MX,87.80,15.85,-97.07,
44,Los Llanos De Aridane,ES,75.20,28.66,-17.92,
51,Fukue,JP,75.24,32.69,128.84,


In [7]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:  
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Omboue,GA,76.17,-1.57,9.26,Hotel Olako
6,Butaritari,KI,82.67,3.07,172.79,Isles Sunset Lodge
9,Mogadishu,SO,78.84,2.04,45.34,Hotel Juba
10,Puerto Del Rosario,ES,78.80,28.50,-13.86,Hotel JM Puerto del Rosario
15,Cabo San Lucas,MX,84.20,22.89,-109.91,Hotel Tesoro Los Cabos
27,Veyrier,CH,80.60,46.17,6.18,Hôtel Métropole Genève
35,Arraial Do Cabo,BR,75.20,-22.97,-42.03,Pousada Porto Praia
41,Puerto Escondido,MX,87.80,15.85,-97.07,Aldea del Bazar Hotel and Spa
44,Los Llanos De Aridane,ES,75.20,28.66,-17.92,Valle Aridane
51,Fukue,JP,75.24,32.69,128.84,Goto Guesthouse Utojuku & SA Car Rental


In [11]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [14]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))